# Patient Like Me - End to End

In [8]:
# !pip install "sagemaker>=2.140.0" "transformers==4.26.1" "datasets[s3]==2.10.1" --upgrade
# !pip install git+https://github.com/huggingface/transformers.git

# !pip install transformers datasets[s3] sagemaker --upgrade
# !pip install scikit-learn
# !pip install accelerate==0.20.3
# !pip install fastprogress
# !pip install ollama
# !ollama pull llama3
# !pip install llama-index==0.10.32
# !pip install langchain
# !pip install langchain_community

In [12]:
!curl -fsSL https://ollama.com/install.sh | sh

curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
>>> Downloading ollama...
curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
######################################################################## 100.0%##O#- #                                                                       
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [18]:
# # run this command on a terminal
!ollama serve
# !ollama pull llama3

2024/07/07 17:49:20 routes.go:1064: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE: OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:1 OLLAMA_MAX_QUEUE:512 OLLAMA_MAX_VRAM:0 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-07-07T17:49:20.139Z level=INFO source=images.go:730 msg="total blobs: 0"
time=2024-07-07T17:49:20.148Z level=INFO source=images.go:737 msg="total unused blobs removed: 0"
time=2024-0

In [5]:
import numpy as np
from sagemaker import get_execution_role
import boto3
import pandas as pd
from io import StringIO # Python 3.
from datasets import load_dataset,Dataset,DatasetDict,concatenate_datasets
from fastprogress.fastprogress import master_bar, progress_bar

import pickle
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd
import json

import matplotlib.pyplot as plt 
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from transformers import TextClassificationPipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pickle
from transformers import AdamW, get_scheduler
from datasets import load_metric
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, average_precision_score
import matplotlib.pyplot as plt 
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score

In [6]:
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-ed-2.2/text_repr.json"

s3 = boto3.resource('s3')
content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df = pd.DataFrame(json_content).T

df['eddischarge'] = [1 if 'admitted' in s.lower() else 0 for s in df['eddischarge']] # admitted = 1, Home = 0
df['medrecon'] = df['medrecon'].fillna("The patient was previously not taking any medications.")
df['pyxis'] = df['pyxis'].fillna("The patient did not receive any medications.")
df['vitals'] = df['vitals'].fillna("The patient had no vitals recorded")
df['codes'] = df['codes'].fillna("The patient received no diagnostic codes")
df = df.drop("admission",axis=1)
df = df.drop("discharge",axis=1)
df = df.drop("eddischarge_category",axis=1)
df['ID'] = df.arrival.astype(str).str.split().str[1].replace(",", " ", regex=True).to_list()
df["patient_info"] = df["arrival"] + " " + df["triage"] + " " + df["medrecon"] + " " + df["vitals"] + " " + df["codes"] + " " + df["pyxis"]
df = df[[col for col in df.columns if col != 'eddischarge'] + ['eddischarge']] # rearrange column to the end
df

,arrival,triage,medrecon,vitals,codes,pyxis,ID,patient_info,eddischarge
33258284,"Patient 10000032, a 52 year old white female, ...","At triage: temperature was 98.4, pulse was 70....",The patient was previously taking the followin...,The patient had the following vitals: At 2180-...,The patient received the following diagnostic ...,The patient did not receive any medications.,10000032,"Patient 10000032, a 52 year old white female, ...",1
38112554,"Patient 10000032, a 52 year old white female, ...","At triage: temperature was 98.9, pulse was 88....",The patient was previously taking the followin...,The patient had the following vitals: At 2180-...,The patient received the following diagnostic ...,The patient received the following medications...,10000032,"Patient 10000032, a 52 year old white female, ...",1
35968195,"Patient 10000032, a 52 year old white female, ...","At triage: temperature was 99.4, pulse was 105...",The patient was previously taking the followin...,The patient had the following vitals: At 2180-...,The patient received the following diagnostic ...,The patient received the following medications...,10000032,"Patient 10000032, a 52 year old white female, ...",1
32952584,"Patient 10000032, a 52 year old white female, ...","At triage: temperature was 97.8, pulse was 87....",The patient was previously taking the followin...,The patient had the following vitals: At 2180-...,The patient received the following diagnostic ...,The patient received the following medications...,10000032,"Patient 10000032, a 52 year old white female, ...",0
39399961,"Patient 10000032, a 52 year old white female, ...","At triage: temperature was 98.7, pulse was 77....",The patient was previously taking the followin...,The patient had the following vitals: At 2180-...,The patient received the following diagnostic ...,The patient received the following medications...,10000032,"Patient 10000032, a 52 year old white female, ...",1
...,...,...,...,...,...,...,...,...,...
34149746,"Patient 19999784, a 57 year old black/african ...","At triage: temperature was 98.8, pulse was 92....",The patient was previously taking the followin...,The patient had the following vitals: At 2119-...,The patient received the following diagnostic ...,The patient received the following medications...,19999784,"Patient 19999784, a 57 year old black/african ...",1
35692999,"Patient 19999784, a 57 year old black/african ...","At triage: temperature was 98.6, pulse was 80....",The patient was previously not taking any medi...,The patient had the following vitals: At 2119-...,The patient received the following diagnostic ...,The patient did not receive any medications.,19999784,"Patient 19999784, a 57 year old black/african ...",1
32917002,"Patient 19999828, a 46 year old white female, ...","At triage: temperature was 96.6, pulse was 112...",The patient was previously taking the followin...,The patient had the following vitals: At 2149-...,The patient received the following diagnostic ...,The patient received the following medications...,19999828,"Patient 19999828, a 46 year old white female, ...",1
30712109,"Patient 19999828, a 46 year old white female, ...","At triage: temperature was 98.1, pulse was 83....",The patient was previously taking the followin...,The patient had the following vitals: At 2147-...,The patient received the following diagnostic ...,The patient received the following medications...,19999828,"Patient 19999828, a 46 year old white female, ...",1


In [9]:
import ollama

In [16]:
import ollama
import requests

def check_ollama_server():
    try:
        response = requests.get("http://localhost:11434/api/version")
        if response.status_code == 200:
            print("Ollama server is running.")
            return True
    except requests.ConnectionError:
        print("Ollama server is not running or not accessible.")
    return False

def get_embeddings():
    if check_ollama_server():
        try:
            result = ollama.embeddings(
                model="llama3",
                prompt="Llamas are members of the camelid family"
            )
            print("Embeddings:", result)
        except Exception as e:
            print(f"Error getting embeddings: {e}")
    else:
        print("Please start the Ollama server before running this script.")

if __name__ == "__main__":
    get_embeddings()

Ollama server is not running or not accessible.
Please start the Ollama server before running this script.
